In [ ]:
# !pip uninstall transformers --yes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers==3.0.2
!pip install datasets

# !git clone https://github.com/jianguoz/Few-Shot-Intent-Detection.git
# !cp -r /content/Few-Shot-Intent-Detection/Datasets/BANKING77 ./BANKING77

     |████████████████████████████████| 769 kB 5.2 MB/s 
     |████████████████████████████████| 3.0 MB 43.1 MB/s 
     |████████████████████████████████| 1.2 MB 41.6 MB/s 
     |████████████████████████████████| 895 kB 48.7 MB/s 
     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 136 kB 38.8 MB/s 
     |████████████████████████████████| 77 kB 5.3 MB/s 
     |████████████████████████████████| 1.1 MB 42.9 MB/s 
     |████████████████████████████████| 212 kB 49.4 MB/s 
     |████████████████████████████████| 127 kB 51.4 MB/s 
     |████████████████████████████████| 144 kB 51.7 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 271 kB 43.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that a

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertConfig, BertModel, BertTokenizer, BertForSequenceClassification
from transformers import AdamW
import random

RANDOM_SEED = 42

torch.manual_seed(RANDOM_SEED)

In [ ]:
def load_examples(file_path, do_lower_case=True):
    examples = []
    
    with open('{}/seq.in'.format(file_path),'r',encoding="utf-8") as f_text, open('{}/label'.format(file_path),'r',encoding="utf-8") as f_label:
        for text, label in zip(f_text, f_label):
            
            e = Inputexample(text.strip(),label=label.strip())
            examples.append(e)
            
    return examples

In [ ]:
class Inputexample(object):
    def __init__(self,text_a,label = None):
        self.text = text_a
        self.label = label

In [ ]:
N = 5
data = []
labels = []
train_samples = []
train_labels = []
embed_dim = 768
batch_size = 12
lr= 1e-4  # you can adjust 
temp = 0.3  # you can adjust 
lamda = 0.01  # you can adjust  
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
path_5s = f'/content/drive/MyDrive/BANKING77/train_10/'
path_test = f'/content/drive/MyDrive/BANKING77/test/'
path_valid = f'/content/drive/MyDrive/BANKING77/valid/'

In [ ]:
train_samples = load_examples(path_5s)
print((train_samples[55].text))
print((train_samples[55].label))

where do i access the auto top-up function?
automatic_top_up


In [ ]:
class CustomTextDataset(Dataset):
    def __init__(self,labels,text,batch_size,repeated_label:bool=False):
        self.labels = labels
        self.text = text
        self.batch_size = batch_size 
        self.count = 0 
        self.repeated_label = repeated_label
        # to use when training with supervise contrastive loss
        if self.repeated_label == True:
            self.used_idx = []
           # write the code here
        #    pass
          

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        
        label = self.labels[idx]
        
        data = self.text[idx]
        
        sample = {"Class": label,"Text": data}


    
        return sample

In [ ]:
# https://downgit.github.io/#/home?url=https:%2F%2Fgithub.com%2Fjianguoz%2FFew-Shot-Intent-Detection%2Ftree%2Fmain%2FDatasets%2FHWU64

# downloading training samples 
train_samples = load_examples(path_5s)

# write code here : for downloading validation samples
valid_samples = load_examples(path_valid)
#write code here : for downloading test samples
test_samples = load_examples(path_test)
# preprocess for training 
for i in range(len(train_samples)):
    data.append(train_samples[i].text)
    labels.append(train_samples[i].label)
    
# write code here :preprocess validation samples
valid_data = []
valid_labels = []
for i in range(len(valid_samples)):
    valid_data.append(valid_samples[i].text)
    valid_labels.append(valid_samples[i].label)
# write code here : preprocess test samples
test_data1 = []
test_labels = []
for i in range(len(valid_samples)):
    test_data1.append(test_samples[i].text)
    test_labels.append(test_samples[i].label)
# dataloader for training 
train_data = CustomTextDataset(labels,data,batch_size=batch_size,repeated_label=True)
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True)


# write code here : dataloader for validation
validation_data = CustomTextDataset(valid_labels,valid_data,batch_size=batch_size,repeated_label=True)
valid_loader = DataLoader(validation_data,batch_size=batch_size,shuffle=True)

# write code here : dataloader for test
test_data = CustomTextDataset(test_labels,test_data1,batch_size=batch_size,repeated_label=True)
test_loader = DataLoader(test_data,batch_size=batch_size,shuffle=True)

# got the number of unique classes from dataset
num_class = len(np.unique(np.array(labels)))

# get text label of uniqure classes
unique_label = np.unique(np.array(labels))

# map text label to index classes
label_maps = {unique_label[i]: i for i in range(len(unique_label))}


In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from datasets import load_metric

from copy import deepcopy
from tqdm.auto import tqdm

def train_model(model, 
                train_loader,
                tokenizer,
                num_epochs,
                optimizer,
                return_best_weight=True,
                valid_loader=None, 
                device=None):
    
    loaders = {"train_loader":train_loader, 
               "valid_loader":valid_loader} if valid_loader else {"train_loader":train_loader}
    
    num_training_steps = num_epochs * np.sum([len(loader) for loader in loaders.values()])
    progress_bar = tqdm(range(num_training_steps))

    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.2)

    metrics = {loader_type:load_metric('accuracy') for loader_type in loaders}
    metrics_history = {loader_type:[] for loader_type in loaders}
    
    best_valid_accuracy = 0
    best_weight = None

    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        print(f'\nEpoch: {epoch+1}', end=" ")
        for loader_type in loaders:

            model.train() if loader_type == 'train_loader' else model.eval()
            for batch in loaders[loader_type]:
                with torch.set_grad_enabled(loader_type == 'train_loader'):

                    sentence = batch["Text"]
                    inputs = tokenizer(sentence,padding=True,truncation=True,return_tensors="pt")
            
                    # move parameter to device
                    inputs = {k:v.to(device) for k,v in inputs.items()}
                    # map string labels to class idex
                    labels = [label_maps[stringtoId] for stringtoId in (batch['Class'])]

                    #print("show out: ",np.unique(labels, return_counts=True))
                    # convert list to tensor
                    labels = torch.tensor(labels).unsqueeze(0)
                    labels = labels.to(device)

                    outputs = model(**inputs,labels=labels)
                    loss = outputs[0]
                    if loader_type == 'train_loader':
                        loss.backward()

                        optimizer.step()
                        optimizer.zero_grad()
                progress_bar.update(1)

                metrics[loader_type].add_batch(predictions=torch.argmax(outputs[1], dim=1), references=labels.view(-1))                
            
            current_accuracy = metrics[loader_type].compute()['accuracy']
            if loader_type == 'valid_loader' and current_accuracy > best_valid_accuracy:
                best_valid_accuracy = current_accuracy
                best_weight = deepcopy(model.state_dict())
            metrics_history[loader_type].append(current_accuracy)

            print(f'{loader_type} accuracy: {current_accuracy:.5f}', end=" ")
            scheduler.step()
    
    if return_best_weight and best_weight is not None:
        print(f'\nLoading best weight @ val_accuracy={best_valid_accuracy:.3f}...')
        model.load_state_dict(best_weight)

    return model, metrics_history

In [ ]:
def freeze_layer(model, last_frozen_layer=None):
    # https://stackoverflow.com/a/65019579/13275449
    
    if last_frozen_layer is None:
        # No freezing from_pretrained
        return 

    for index, (name, param) in enumerate(list(model.named_parameters())[:last_frozen_layer+1]):      
        param.requires_grad = False

    assert all([param.requires_grad == False for name, param in list(model.named_parameters())[:last_frozen_layer+1]]), f'Not all layers before layer {last_frozen_layer+1} is freezed'
    assert all([param.requires_grad == True for name, param in list(model.named_parameters())[last_frozen_layer+1:]]), f'Not all layers before after {last_frozen_layer+1} is trainable'

In [ ]:
num_epochs = 50


MODEL_NAMES = ['bert-base-uncased']

for model_name in MODEL_NAMES:
    print(model_name)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=False)    
    config = BertConfig.from_pretrained('bert-base-uncased', num_labels=77)
    # special_tokens_dict = {'eos_token': '[eos]'}
    # num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

    # state_dict = torch.load("/content/drive/MyDrive/ubuntu25_bert.pt")
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)
    # model.resize_token_embeddings(len(tokenizer))
    # model.bert.load_state_dict(state_dict=state_dict)
    
    freeze_layer(model, last_frozen_layer=80)
    model = model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    trained_model, metrics_history = train_model(model=model, 
                                                train_loader=train_loader, 
                                                tokenizer=tokenizer,
                                                num_epochs=num_epochs,
                                                optimizer=optimizer,
                                                valid_loader=valid_loader, 
                                                device=device)
    torch.save(trained_model.state_dict(), './BERT_FP_finetuned.pt')

bert-base-uncased


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  0%|          | 0/9700 [00:00<?, ?it/s]


Epoch: 1 train_loader accuracy: 0.01169 valid_loader accuracy: 0.04481 
Epoch: 2 train_loader accuracy: 0.15974 valid_loader accuracy: 0.33636 
Epoch: 3 train_loader accuracy: 0.43896 valid_loader accuracy: 0.58052 
Epoch: 4 train_loader accuracy: 0.71169 valid_loader accuracy: 0.69610 
Epoch: 5 train_loader accuracy: 0.84545 valid_loader accuracy: 0.74351 
Epoch: 6 train_loader accuracy: 0.95195 valid_loader accuracy: 0.78701 
Epoch: 7 train_loader accuracy: 0.97013 valid_loader accuracy: 0.81623 
Epoch: 8 train_loader accuracy: 0.98182 valid_loader accuracy: 0.82013 
Epoch: 9 train_loader accuracy: 0.99740 valid_loader accuracy: 0.82857 
Epoch: 10 train_loader accuracy: 0.99870 valid_loader accuracy: 0.82727 
Epoch: 11 train_loader accuracy: 1.00000 valid_loader accuracy: 0.82532 
Epoch: 12 train_loader accuracy: 1.00000 valid_loader accuracy: 0.82338 
Epoch: 13 train_loader accuracy: 1.00000 valid_loader accuracy: 0.82468 
Epoch: 14 train_loader accuracy: 1.00000 valid_loader accur